## Step 1 : Construction

Build a question_reference list for further evaluatuion

In [5]:
import pandas as pd

ds = pd.read_parquet("./rag-test-dataset-calm-adrd.parquet")

ds.head()

,Question,Context,Answer,calm-adrd,gpt-4o-mini,gpt-4o,References,Generation
0,Can you recommend activities that are suitable...,Content: What are some tips for Finding Activi...,When looking for activities for someone with d...,I understand that finding suitable activities ...,Absolutely! Engaging individuals with dementia...,Engaging activities for individuals with demen...,[https://www.agingcare.com/articles/my-elderly...,When looking for activities for someone with d...
1,What are the benefits of hospice care for indi...,Content: How to be an effective caregiver in t...,Hospice care offers several benefits for indiv...,I understand that navigating the advanced stag...,Hospice care can provide numerous benefits for...,Hospice care can provide numerous benefits for...,[https://www.caregiver.org/resource/alzheimers...,Hospice care offers several benefits for indiv...
2,How can I navigate difficult family dynamics w...,Content: How to Handle Seemingly Manipulative ...,When navigating difficult family dynamics whil...,Navigating difficult family dynamics while car...,Caring for a loved one with Alzheimer's can be...,Navigating difficult family dynamics while car...,[https://www.agingcare.com/articles/dementia-b...,When navigating difficult family dynamics whil...
3,What are the early signs of dementia?,Content: None The early warning signs of demen...,The early signs of dementia can vary from pers...,I understand that learning about the early sig...,"Early signs of dementia can vary, but here are...",Early signs of dementia can vary between indiv...,[https://www.agingcare.com/articles/alzheimers...,The early signs of dementia can vary from pers...
4,"How does palliative care differ from hospice, ...",Content: What end-of-life care options are ava...,Palliative care and hospice care are both form...,I understand that navigating care options for ...,Palliative care and hospice care are both appr...,Palliative care and hospice care are both appr...,[https://www.agingcare.com/articles/end-stage-...,Palliative care and hospice care are both form...


In [6]:
## Constructing Question list

question_list = ds["Question"].tolist()

question_list[:10]



['Can you recommend activities that are suitable for someone with dementia to engage in and enjoy?',
 "What are the benefits of hospice care for individuals in the advanced stages of Alzheimer's disease?",
 "How can I navigate difficult family dynamics when caring for a loved one with Alzheimer's?",
 'What are the early signs of dementia?',
 'How does palliative care differ from hospice, and how can it help improve the quality of life for someone with advanced dementia?',
 "How can I ensure my loved one's safety at home, especially if they have a tendency to wander or become agitated?",
 'What activities are suitable for people with dementia, and how can I encourage participation?',
 'Are there any risk factors that can contribute to developing delirium?',
 'Are certain types of anesthesia more likely to cause delirium?',
 "How do I navigate legal issues related to caring for a loved one with Alzheimer's, such as power of attorney and guardianship?"]

In [7]:
## Prepare for parsing

import pprint
from typing import TypedDict, List

class Reference(TypedDict):
    source: str
    content: str

example_reference = """
Content: What are some tips for Finding Activities to Do with Dementia Patients? Consider the person’s interests.    The best place to start when brainstorming activities that your loved one might enjoy is taking inventory of their past and current interests and hobbies. Examples might include gardening, crafting, playing or watching sports, cooking, reading about historical events, fishing, photography, or listening to or playing certain types of music. Once you’ve made a list of potential ideas, it’s time to get creative.   Hobbies often fall by the wayside when a senior experiences increasing physical and/or mental challenges. Beth Kallmyer, former Director of Family and Information Services for the Alzheimer’s Association, recommends adjusting these pastimes to fit their abilities. Providing additional assistance and supervision is often necessary with dementia patients, but as their condition progresses, adapting the activities themselves is also required.
Source: [1] <https://www.agingcare.com/articles/my-elderly-parent-has-dementia-what-activities-can-we-do-together-137087.htm>
Content: What are some easily-modifiable activities that may be good for seniors with Alzheimer's? Stick to simple pleasures.   Keeping things simple and straightforward is often the best course of action when coming up with activities for people with Alzheimer's. Going to a local park to feed the birds and fish is an easy task that may be very fun for a person with Alzheimer's. You can also take turns reading a favorite book aloud. This can be an easy way to encourage a senior to exercise their mind while giving them the pleasure of reading a beloved book. Listen to music.   Research has shown that listening to music can help a person with Alzheimer's remember events, people, and places from their past. Additionally, music can be a way to get a senior moving through dance or song. Music can be themed to match your needs; sing along activities, upbeat motivating tunes, or relaxing sounds may all play a part in your day. Cook and clean.
Source: [2] <https://www.agingcare.com/articles/activities-for-alzheimers-148840.htm>
Content: What are some activities for Seniors with Dementia? “Activities depend on the senior’s level of dementia. Some people are still able to do things, like read, do puzzles, watch TV, tend to plants, etc., but some are not. I found that unless there is a person leading, directing and involving the patient, they may not have the insight or motivation to engage in an activity. There are activity boards that you can purchase that sit in the person’s lap or on a table and have various tasks on them, like buttons, zippers, and Velcro, and include things that have different textures and even make sounds. They are designed specifically for people with dementia and are aimed at keeping a person's hands busy and providing mental stimulation.” –Sunnygirl1
Source: [3] <https://www.agingcare.com/articles/hobbies-and-activities-for-seniors-tips-from-family-caregivers-205881.htm>
"""

def parse_context_column(context: str) -> List[Reference]:
    """Parse the context column to extract content and source.
    
    Args:
        context: The context string containing content and source information
        
    Returns:
        List[Reference]: A list of dictionaries with content and source fields
    """
    # Split the context by lines
    lines = context.strip().split('\n')
    
    references = []
    content_parts = []
    source = ""
    
    for line in lines:
        if line.startswith("Content:"):
            # If we already have content and are starting a new content section,
            # save the previous reference
            if content_parts and (source or "None" in " ".join(content_parts)):
                reference = Reference(
                    content=" ".join(content_parts),
                    source=source
                )
                references.append(reference)
                content_parts = []
                source = ""
            
            # Remove the "Content: " prefix and add to content parts
            content_parts.append(line[len("Content:"):].strip())
        elif line.startswith("Source:"):
            # Extract only the URL part from the source line
            source_line = line[len("Source:"):].strip()
            # Find the URL within angle brackets
            import re
            url_match = re.search(r'<(.*?)>', source_line)
            if url_match:
                source = url_match.group(1)
    
    # Add the last reference if there's content
    if content_parts:
        reference = Reference(
            content=" ".join(content_parts),
            source=source
        )
        references.append(reference)
    
    # If no references were found, return a single empty reference
    if not references:
        references.append(Reference(source="", content=""))
    
    return references

exp_res = parse_context_column(example_reference)

print(type(exp_res))
pprint.pprint(exp_res, indent = 4)


<class 'list'>
[   {   'content': 'What are some tips for Finding Activities to Do with '
                   'Dementia Patients? Consider the person’s interests.    The '
                   'best place to start when brainstorming activities that '
                   'your loved one might enjoy is taking inventory of their '
                   'past and current interests and hobbies. Examples might '
                   'include gardening, crafting, playing or watching sports, '
                   'cooking, reading about historical events, fishing, '
                   'photography, or listening to or playing certain types of '
                   'music. Once you’ve made a list of potential ideas, it’s '
                   'time to get creative.   Hobbies often fall by the wayside '
                   'when a senior experiences increasing physical and/or '
                   'mental challenges. Beth Kallmyer, former Director of '
                   'Family and Information Services for th

In [8]:
# Construct reference (context) list according to question

from typing import TypedDict, List

class QuestionReference(TypedDict):
    question: str
    references: list[Reference]
    
reference_list: List[QuestionReference] = []

for idx, row in ds.iterrows():
    question = row["Question"]
    references: List[Reference] = parse_context_column(row["Context"])
    
    reference_list.append(QuestionReference(question = question, references = references))

reference_list[0:3]

[{'question': 'Can you recommend activities that are suitable for someone with dementia to engage in and enjoy?',
  'references': [{'content': 'What are some tips for Finding Activities to Do with Dementia Patients? Consider the person’s interests.    The best place to start when brainstorming activities that your loved one might enjoy is taking inventory of their past and current interests and hobbies. Examples might include gardening, crafting, playing or watching sports, cooking, reading about historical events, fishing, photography, or listening to or playing certain types of music. Once you’ve made a list of potential ideas, it’s time to get creative.   Hobbies often fall by the wayside when a senior experiences increasing physical and/or mental challenges. Beth Kallmyer, former Director of Family and Information Services for the Alzheimer’s Association, recommends adjusting these pastimes to fit their abilities. Providing additional assistance and supervision is often necessary w

## Step 2: Get answer from CaLM

Now we have `reference_list` as a gold standard data to compare with, next step is to get answers from CaLM AI ADRD

In [11]:
calm_reference_list = []

for idx, row in ds.iterrows():
    calm_ref = row['References'].tolist()
    calm_reference_list.append(calm_ref)

In [ ]:
## Calculate recall between reference_list and calm_reference_list for each question

from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

def calculate_reference_recall(gold_references, model_references):
    """
    计算参考文献的召回率
    
    Args:
        gold_references: 标准参考文献列表
        model_references: 模型生成的参考文献列表
    
    Returns:
        平均召回率
    """
    recalls = []
    
    for i, (gold_ref_item, model_ref_list) in enumerate(zip(reference_list, calm_reference_list)):
        gold_urls = set()
        for ref in gold_ref_item['references']:
            if 'source' in ref:
                gold_urls.add(ref['source'])
        
        model_urls = set(model_ref_list)
        
        # 计算召回率
        if len(gold_urls) > 0:
            recall = len(gold_urls.intersection(model_urls)) / len(gold_urls)
        else:
            recall = 0.0
            
        recalls.append(recall)
        
        print(f"问题 {i+1}: 召回率 = {recall:.4f}")
        print(f"  - 标准参考文献: {gold_urls}")
        print(f"  - 模型参考文献: {model_urls}")
        print()
    
    avg_recall = np.mean(recalls)
    print(f"平均召回率: {avg_recall:.4f}")
    
    return avg_recall

# 计算召回率
avg_recall = calculate_reference_recall(reference_list, calm_reference_list)
